# **LP1_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

1. Variables for the volume of chemical components in each product (measured in litres):

   C1_GCA: Volume of component C1 in product GCA
   C2_GCA: Volume of component C2 in product GCA
   C1_GCB: Volume of component C1 in product GCB
   C2_GCB: Volume of component C2 in product GCB
   C1_GCC: Volume of component C1 in product GCC
   C2_GCC: Volume of component C2 in product GCC

2. Variables for the volume of each product (measured in litres):

   V_GCA: Volume of product GCA
   V_GCB: Volume of product GCB
   V_GCC: Volume of product GCC

3. Variables for the volume of the critical element in each product (measured in litres):

   CE_GCA: Volume of the critical element in product GCA
   CE_GCB: Volume of the critical element in product GCB
   CE_GCC: Volume of the critical element in product GCC


In total, the problem involves 9 variables, which are to be optimized to maximize the total profit of the firm while meeting the constraints on minimum requirements, maximum availability,

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
print(response2.choices[0].message.content)

The objective function to maximize the profit can be represented as:

Maximize Profit = 120*V_GCA + 135*V_GCB + 155*V_GCC


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1. Supply Constraints for components:
   C1_GCA + C1_GCB + C1_GCC <= 10,000
   C2_GCA + C2_GCB + C2_GCC <= 15,000

2. Demand Constraints for end products:
   V_GCA >= 6,000
   V_GCB >= 7,000
   V_GCC >= 9,000

3. Component volume Constraints (as there is no loss or gain in mixing):
   C1_GCA + C2_GCA = V_GCA
   C1_GCB + C2_GCB = V_GCB
   C1_GCC + C2_GCC = V_GCC

4. Critical Element Constraints:
   0.4*C1_GCA + 0.2*C2_GCA >= 0.3*V_GCA
   0.4*C1_GCB + 0.2*C2_GCB <= 0.3*V_GCB

5. Minimum ratio of C1 with C2 in product GCC:
   C1_GCC >= 0.3*C2_GCC

All the variables: C1_GCA, C2_GCA, C1_GCB, C2_GCB, C1_GCC, C2_GCC, V_GCA, V_GCB, V_GCC must be greater than or equal to zero, as negative volume wouldn't make sense in this context.


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [12]:
Markdown(response4.choices[0].message.content)

Here is the Python Pyomo code for the given optimization problem:

```python
#Importing necessary modules
from pyomo.environ import *

#Creating a Concrete model
model = ConcreteModel()

#Creating the variables
model.C1_GCA = Var(within=NonNegativeReals)
model.C2_GCA = Var(within=NonNegativeReals)

model.C1_GCB = Var(within=NonNegativeReals)
model.C2_GCB = Var(within=NonNegativeReals)

model.C1_GCC = Var(within=NonNegativeReals)
model.C2_GCC = Var(within=NonNegativeReals)

model.V_GCA = Var(within=NonNegativeReals)
model.V_GCB = Var(within=NonNegativeReals)
model.V_GCC = Var(within=NonNegativeReals)

#Defining the objective function
model.profit = Objective(expr = 120*model.V_GCA + 135*model.V_GCB + 155*model.V_GCC, 
                         sense = maximize)

#Defining the constraints
model.supplyC1 = Constraint(expr = model.C1_GCA + model.C1_GCB + model.C1_GCC <= 10000)
model.supplyC2 = Constraint(expr = model.C2_GCA + model.C2_GCB + model.C2_GCC <= 15000)

model.demandGCA = Constraint(expr = model.V_GCA >= 6000)
model.demandGCB = Constraint(expr = model.V_GCB >= 7000)
model.demandGCC = Constraint(expr = model.V_GCC >= 9000)

model.mixGCA = Constraint(expr = model.C1_GCA + model.C2_GCA == model.V_GCA)
model.mixGCB = Constraint(expr = model.C1_GCB + model.C2_GCB == model.V_GCB)
model.mixGCC = Constraint(expr = model.C1_GCC + model.C2_GCC == model.V_GCC)

model.criticalElementGCA = Constraint(expr = 0.4*model.C1_GCA + 0.2*model.C2_GCA >= 0.3*model.V_GCA)
model.criticalElementGCB = Constraint(expr = 0.4*model.C1_GCB + 0.2*model.C2_GCB <= 0.3*model.V_GCB)

model.minRatioGCC = Constraint(expr = model.C1_GCC >= 0.3*model.C2_GCC)

#Solving the model
SolverFactory('glpk').solve(model)
model.display()
```

Sample data are referred in the constraints, we have used 10000 and 15000 as the supply constraint for respective raw materials, 6000, 7000, and 9000 as the demand constraint for the respective final products, and the prices for the respective end products as 120, 135, and 155. 

This is a sample linear programming problem which can be solved using 'glpk' or 'cbc' solver in Pyomo. The problem involves maximizing the profit under given constraints.

### **Run the code Generated by GPT4**

In [15]:
from pyomo.environ import *

# Creating a Concrete model
model = ConcreteModel()

# Creating the variables
model.C1_GCA = Var(within=NonNegativeReals)
model.C2_GCA = Var(within=NonNegativeReals)

model.C1_GCB = Var(within=NonNegativeReals)
model.C2_GCB = Var(within=NonNegativeReals)

model.C1_GCC = Var(within=NonNegativeReals)
model.C2_GCC = Var(within=NonNegativeReals)

model.V_GCA = Var(within=NonNegativeReals)
model.V_GCB = Var(within=NonNegativeReals)
model.V_GCC = Var(within=NonNegativeReals)

# Defining the objective function
model.profit = Objective(expr = 120*model.V_GCA + 135*model.V_GCB + 155*model.V_GCC,
                         sense = maximize)

# Defining the constraints
model.supplyC1 = Constraint(expr = model.C1_GCA + model.C1_GCB + model.C1_GCC <= 10000)
model.supplyC2 = Constraint(expr = model.C2_GCA + model.C2_GCB + model.C2_GCC <= 15000)

model.demandGCA = Constraint(expr = model.V_GCA >= 6000)
model.demandGCB = Constraint(expr = model.V_GCB >= 7000)
model.demandGCC = Constraint(expr = model.V_GCC >= 9000)

model.mixGCA = Constraint(expr = model.C1_GCA + model.C2_GCA == model.V_GCA)
model.mixGCB = Constraint(expr = model.C1_GCB + model.C2_GCB == model.V_GCB)
model.mixGCC = Constraint(expr = model.C1_GCC + model.C2_GCC == model.V_GCC)

model.criticalElementGCA = Constraint(expr = 0.4*model.C1_GCA + 0.2*model.C2_GCA >= 0.3*model.V_GCA)
model.criticalElementGCB = Constraint(expr = 0.4*model.C1_GCB + 0.2*model.C2_GCB <= 0.3*model.V_GCB)

model.minRatioGCC = Constraint(expr = model.C1_GCC >= 0.3*model.C2_GCC)

# Solving the model
SolverFactory('glpk').solve(model)
model.display()


Model unknown

  Variables:
    C1_GCA : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 3000.0 :  None : False : False : NonNegativeReals
    C2_GCA : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 3000.0 :  None : False : False : NonNegativeReals
    C1_GCB : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    C2_GCB : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 7000.0 :  None : False : False : NonNegativeReals
    C1_GCC : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 7000.0 :  None : False : False : NonNegativeReals
    C2_GCC : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 5000.0 :  None :

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**